In [1]:
!pip install -qU langchain langgraph langchain_openai reportlab numexpr python-dotenv


In [2]:
!pip install -qU openinference-instrumentation-langchain 


In [3]:
# Import required libraries
import os
import operator
from typing import TypedDict, Annotated, List
import json

from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END

# For PDF Generation
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

# For Calculator
import numexpr

# Load API keys (optional, assumes OPENAI_API_KEY is set in environment)
from dotenv import load_dotenv
load_dotenv()
print("OPENAI_API_KEY loaded:", os.getenv("OPENAI_API_KEY") is not None)

OPENAI_API_KEY loaded: True


In [4]:
from phoenix.otel import register

In [5]:
tracer_provider = register(
  project_name="langgraph", # Default is 'default'
  auto_instrument=True # Auto-instrument your app based on installed OI dependencies
)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: langgraph
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [6]:
@tool
def calculator(expression: str) -> str:
    """
    Evaluates a simple mathematical expression string (e.g., "2 * (3 + 5)").
    Args:
        expression: The mathematical expression string to evaluate.
    Returns:
        The numerical result as a string.
    """
    # Directly evaluate, assuming input is a valid string expression for numexpr
    # No error handling - will raise exception on invalid input
    result = numexpr.evaluate(expression).item()
    return str(result) # Return the result directly as a string

In [7]:
@tool
def save_to_pdf(content: str, filename: str = "output.pdf") -> str:
    """
    Saves the given text content to a PDF file in the '{PDF_OUTPUT_DIR}' directory.
    Use this tool *only* when the user explicitly asks to save the previous conversation or answer as a PDF.
    Args:
        content: The text content to save.
        filename: The desired name for the PDF file (e.g., 'summary.pdf'). Defaults to 'output.pdf'.
    Returns:
        A confirmation message or an error message.
    """
    PDF_OUTPUT_DIR = "."
    print(f"🛠️ Calling Save to PDF Tool with filename: {filename}")
    try:
        # Ensure filename has .pdf extension
        if not filename.lower().endswith(".pdf"):
            filename += ".pdf"
        
        # Sanitize filename (basic example, consider more robust sanitization)
        filename = os.path.basename(filename) # Prevent directory traversal
        filepath = os.path.join(PDF_OUTPUT_DIR, filename)

        doc = SimpleDocTemplate(filepath, pagesize=letter)
        styles = getSampleStyleSheet()
        # Replace newlines with <br/> tags for paragraph breaks in PDF
        story = [Paragraph(content.replace('\n', '<br/>'), styles['Normal'])]
        doc.build(story)
        return f"Content successfully saved to {filepath}"
    except Exception as e:
        return f"Error saving content to PDF '{filename}': {e}"

# List of tools available to the agent
tools = [calculator, save_to_pdf]

# Helper: Create a mapping from tool name to tool function
tool_map = {tool.name: tool for tool in tools}

In [8]:
# Define the state for our graph
class AgentState(TypedDict):
    # Messages queue: Stores the conversation history (Human, AI, Tool messages)
    # operator.add makes it so new messages are appended to the list
    messages: Annotated[List[BaseMessage], operator.add]

In [9]:
# --- LLM and Tool Binding ---
# Use a model that supports tool calling
# Make sure OPENAI_API_KEY is set in your environment
try:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) 
    # Bind tools to LLM
    llm_with_tools = llm.bind_tools(tools)
    print("LLM initialized and tools bound.")
except Exception as e:
    print(f"Error initializing LLM. Check API key and model availability: {e}")
    llm_with_tools = None # Set to None to prevent errors later if init fails

# --- Node Functions ---

# 1. Agent Node: Calls the LLM to decide the next action or generate a response
def agent_node(state: AgentState):
    """Invokes the LLM to get the next step or response."""
    print("---AGENT NODE---")
    if llm_with_tools is None:
         raise ValueError("LLM not initialized. Cannot proceed.")
    # The LLM's response is based on the current message history
    response = llm_with_tools.invoke(state['messages'])
    print(f"LLM Response: {response.content}")
    if response.tool_calls:
        print(f"LLM requested tools: {response.tool_calls}")
    # The response (AIMessage with potential tool_calls) is added to the state
    # by LangGraph thanks to the `operator.add` in AgentState definition
    return {"messages": [response]}

# 2. Tool Node: Executes the tools called by the LLM
def tool_node(state: AgentState):
    """Executes tools requested by the LLM and returns their outputs."""
    print("---TOOL NODE---")
    last_message = state['messages'][-1] # Get the latest message (should be AIMessage with tool calls)

    # Check if the last message is an AIMessage and has tool calls
    if not isinstance(last_message, AIMessage) or not last_message.tool_calls:
        print("Warning: Tool node called without tool calls in the last message.")
        return {} # No tools to execute

    tool_messages = []
    print(f"Executing tools: {[tc['name'] for tc in last_message.tool_calls]}")

    for tool_call in last_message.tool_calls:
        tool_name = tool_call['name']
        if tool_name in tool_map:
            selected_tool = tool_map[tool_name]
            tool_input = tool_call['args']

            print(f"  - Calling tool '{tool_name}' with args: {tool_input}")
            try:
                # Invoke the tool. .invoke() handles both regular functions and Runnables
                tool_output = selected_tool.invoke(tool_input)
                print(f"  - Tool '{tool_name}' output: {tool_output}")
            except Exception as e:
                print(f"  - Error executing tool {tool_name}: {e}")
                tool_output = f"Error executing tool {tool_name}: {e}"

            # Create a ToolMessage with the output and add it to our list
            tool_messages.append(
                ToolMessage(content=str(tool_output), tool_call_id=tool_call['id'])
            )
        else:
            print(f"  - Warning: Tool '{tool_name}' not found.")
            # Still add a ToolMessage indicating the error
            tool_messages.append(
                ToolMessage(content=f"Error: Tool '{tool_name}' not found.", tool_call_id=tool_call['id'])
            )

    # Return the list of ToolMessages to be appended to the state
    return {"messages": tool_messages}

LLM initialized and tools bound.


In [10]:
def router(state: AgentState) -> str:
    """Decides the next step based on the last message."""
    print("---ROUTER---")
    last_message = state['messages'][-1]

    # Check 1 : Is the last message an AIMessage with tool calls?
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        print("Routing: Agent requested tools -> tool_node")
        return "tool_node" # Route to the tool node to execute tools

    # Check 2 : Is the last message a ToolMessage?
    if isinstance(last_message, ToolMessage):
         print("Routing: Tools executed -> agent_node")
         return "agent_node" # Route back to the agent node to process tool results

    # Check 3 : Is the last message an AIMessage *without* tool calls?
    if isinstance(last_message, AIMessage) and not last_message.tool_calls:
        print("Routing: Agent provided final answer for this turn -> end")
        return "end" # End the current flow, wait for next user input

    # Default/Fallback (should ideally not be hit often in this structure)
    print("Routing: Fallback -> end")
    return "end"

In [11]:
# Create a new StateGraph with our defined AgentState
workflow = StateGraph(AgentState)

# Add the nodes to the graph
workflow.add_node("agent_node", agent_node)
workflow.add_node("tool_node", tool_node)

# Set the entry point: The first step after user input is always the agent node
workflow.set_entry_point("agent_node")

# Add conditional edges based on the router's decision
workflow.add_conditional_edges(
    "agent_node",  # Start node
    router,        # Function that decides the next node
    {
        "tool_node": "tool_node", # If router returns "tool_node", go to tool_node
        "end": END,               # If router returns "end", finish the graph execution
        # Note: We don't need agent_node here as router logic doesn't loop back immediately
    }
)

# Add a regular edge from the tool_node back to the agent_node
# After tools are executed, the agent needs to process their results
workflow.add_edge("tool_node", "agent_node")

# Compile the graph into a runnable application
try:
    app = workflow.compile()
    print("Graph compiled successfully!")
    # Optional: Visualize the graph (requires graphviz)
    # from IPython.display import Image, display
    # try:
    #     display(Image(app.get_graph().draw_mermaid_png()))
    # except Exception as e:
    #     print(f"Graph visualization failed (is graphviz installed?): {e}")
except Exception as e:
    print(f"Error compiling graph: {e}")
    app = None



Graph compiled successfully!


In [12]:
if app is None:
    print("Graph compilation failed. Cannot run the agent.")
else:
    print("\n--- Starting Agent Interaction (type 'quit' or 'exit' to stop) ---")
    conversation_history = [] # Start with an empty history for each run

    while True:
        # Modified prompt
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit"]:
            print("Exiting agent interaction.")
            break

        if not user_input.strip():
            continue

        # Add user message to history
        new_human_message = HumanMessage(content=user_input)
        # Append to existing history for this session
        conversation_history.append(new_human_message)

        # Prepare the input state for the graph invocation
        # We pass the *entire* current conversation history
        initial_state = {
            "messages": conversation_history
        }

        print("\n--- Running Graph ---")
        # Use stream to observe the flow step-by-step
        # Set a recursion limit for safety
        events = app.stream(initial_state, {"recursion_limit": 15})

        final_state_messages = []
        # Track messages added *during this specific graph run*
        messages_from_this_run = []

        for event in events:
            step_name = list(event.keys())[0]

            if step_name != "__end__":
                # Accumulate messages from intermediate steps of *this* run
                if "messages" in event[step_name]:
                      messages_from_this_run.extend(event[step_name]["messages"])

            # Check for the final state when the graph naturally ends for this turn
            if "__end__" in event:
                final_state_data = event["__end__"]
                # The final state contains the *complete* message list up to this point
                final_state_messages = final_state_data.get("messages", [])
                print("--- Graph Execution Finished ---")
                break # Stop processing events for this turn

        # Update conversation history with the final list from the graph run
        # If the graph finished properly, final_state_messages has the full history
        # If it somehow stopped early, we might need to manually combine (less ideal)
        conversation_history = final_state_messages if final_state_messages else conversation_history + messages_from_this_run


        # Print the *last* AI response from this turn
        last_ai_response = next((msg for msg in reversed(conversation_history) if isinstance(msg, AIMessage)), None)

        if last_ai_response:
            print("\nAssistant:")
            if last_ai_response.content:
                print(last_ai_response.content)
            if last_ai_response.tool_calls:
                 print(f"(Debug: Requested tools: {last_ai_response.tool_calls})")
        else:
            # This might happen if the graph ended without producing an AI message
            print("\nAssistant: (No new response generated for this turn)")

        print("\n" + "="*50 + "\n")

    print("--- Agent Interaction Ended ---")


--- Starting Agent Interaction (type 'quit' or 'exit' to stop) ---


KeyboardInterrupt: Interrupted by user